# Feature Selection

## Merging Dataframes

In [1]:
import pandas as pd
import requests
from io import BytesIO
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
pd.set_option('display.max_columns', None)


In [35]:
## testing remote opening of files
# constructor of google download links
dl_construct = 'https://drive.google.com/uc?export=download&id='
# id from share link google drive
file_id = '1zUzr93toblHkH2f3Zj5CEuD81qND7WYd'
csv = requests.get(f'{dl_construct}{file_id}').content
memory = BytesIO(csv)
df = pd.read_csv(memory)
df.shape

(8669, 110)

In [36]:
df

,Unnamed: 0,Link_ID,linkid,Daily_Average_Traffic_Intensity,Average_Velocity_of_Vehicle_Traffic,Median_of_velocity_of_Vehicle_Traffic,FirstQuartil_of_velocity_of_Vehicle_Traffic,ThirdQuartil_of_velocity_of_Vehicle_Traffic,Func_Class,Speed_Cat,geometry,feat_barrier_block,feat_barrier_bollard,feat_barrier_border_control,feat_barrier_bump_gate,feat_barrier_cattle_grid,feat_barrier_chain,feat_barrier_cycle_barrier,feat_barrier_debris,feat_barrier_door,feat_barrier_entrance,feat_barrier_fence,feat_barrier_gate,feat_barrier_height_restrictor,feat_barrier_jersey_barrier,feat_barrier_kerb,feat_barrier_kissing_gate,feat_barrier_lift_gate,feat_barrier_rope,feat_barrier_sally_port,feat_barrier_security_check,feat_barrier_stile,feat_barrier_swing_gate,feat_barrier_toll_booth,feat_barrier_toll_gantry,feat_barrier_turnstile,feat_barrier_wall,feat_barrier_yes,feat_highway_bus_stop,feat_highway_crossing,feat_highway_crossing;traffic_signals,feat_highway_elevator,feat_highway_footway,feat_highway_give_way,feat_highway_milestone,feat_highway_mini_roundabout,feat_highway_motorway_junction,feat_highway_path,feat_highway_platform,feat_highway_raceway,feat_highway_rest_area,feat_highway_services,feat_highway_speed_camera,feat_highway_steps,feat_highway_stop,feat_highway_street_lamp,feat_highway_toll_gantry,feat_highway_traffic_mirror,feat_highway_traffic_signals,feat_highway_traffic_signals;crossing,feat_highway_trailhead,feat_highway_turning_circle,feat_highway_turning_loop,feat_man_made_advertising,feat_man_made_antenna,feat_man_made_beacon,feat_man_made_bunker_silo,feat_man_made_cairn,feat_man_made_chimney,feat_man_made_communications_tower,feat_man_made_crane,feat_man_made_cross,feat_man_made_flagpole,feat_man_made_gasometer,feat_man_made_kiln,feat_man_made_lighthouse,feat_man_made_lookout_tower,feat_man_made_manhole,feat_man_made_mast,feat_man_made_monitoring_station,feat_man_made_pier,feat_man_made_pole,feat_man_made_reservoir,feat_man_made_reservoir_covered,feat_man_made_satellite_centre,feat_man_made_silo,feat_man_made_standpipe,feat_man_made_storage_tank,feat_man_made_street_cabinet,feat_man_made_surveillance,feat_man_made_survey_point,feat_man_made_tanks,feat_man_made_telescope,feat_man_made_threshing_floor,feat_man_made_tower,feat_man_made_utility_pole,feat_man_made_ventilation_shaft,feat_man_made_wastewater_plant,feat_man_made_water_tank,feat_man_made_water_tap,feat_man_made_water_tower,feat_man_made_water_well,feat_man_made_water_works,feat_man_made_watermill,feat_man_made_windmill,feat_man_made_windpump,feat_man_made_works,feat_man_made_yes,points,sum_points
0,0,80216819,80216819,6224.778569,45.208716,44.464286,28.000000,59.000000,2,6,"MULTILINESTRING ((-9.164020000000001 38.7703, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[<shapely.geometry.point.Point object at 0x160...,5
1,1,80216858,80216858,2236.054168,79.563308,82.750000,71.750000,93.250000,2,4,"MULTILINESTRING ((-9.166449999999999 38.74274,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[<shapely.geometry.point.Point object at 0x160...,18
2,2,80216859,80216859,2138.725039,65.955069,67.333333,61.000000,75.000000,3,6,"MULTILINESTRING ((-9.166449999999999 38.74274,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0

In [39]:
(df == 0).all()

Unnamed: 0                             False
Link_ID                                False
linkid                                 False
Daily_Average_Traffic_Intensity        False
Average_Velocity_of_Vehicle_Traffic    False
                                       ...  
feat_man_made_windpump                  True
feat_man_made_works                     True
feat_man_made_yes                       True
points                                 False
sum_points                             False
Length: 110, dtype: bool

In [40]:
df.max()

Unnamed: 0                                                                         17686
Link_ID                                                                       1223525692
linkid                                                                        1223525692
Daily_Average_Traffic_Intensity                                             26714.250016
Average_Velocity_of_Vehicle_Traffic                                          3983.178775
                                                             ...                        
feat_man_made_windpump                                                          0.000000
feat_man_made_works                                                             0.000000
feat_man_made_yes                                                               0.000000
points                                 [<shapely.geometry.point.Point object at 0x160...
sum_points                                                                            97
Length: 110, dtype: o

In [6]:
columns=['Link_ID','Unnamed: 0','geometry','points']

In [7]:
df.drop(columns=columns, inplace=True)
df

,linkid,Daily_Average_Traffic_Intensity,Average_Velocity_of_Vehicle_Traffic,Median_of_velocity_of_Vehicle_Traffic,FirstQuartil_of_velocity_of_Vehicle_Traffic,ThirdQuartil_of_velocity_of_Vehicle_Traffic,Func_Class,Speed_Cat,feat_barrier_block,feat_barrier_bollard,feat_barrier_border_control,feat_barrier_bump_gate,feat_barrier_cattle_grid,feat_barrier_chain,feat_barrier_cycle_barrier,feat_barrier_debris,feat_barrier_door,feat_barrier_entrance,feat_barrier_fence,feat_barrier_gate,feat_barrier_height_restrictor,feat_barrier_jersey_barrier,feat_barrier_kerb,feat_barrier_kissing_gate,feat_barrier_lift_gate,feat_barrier_rope,feat_barrier_sally_port,feat_barrier_security_check,feat_barrier_stile,feat_barrier_swing_gate,feat_barrier_toll_booth,feat_barrier_toll_gantry,feat_barrier_turnstile,feat_barrier_wall,feat_barrier_yes,feat_highway_bus_stop,feat_highway_crossing,feat_highway_crossing;traffic_signals,feat_highway_elevator,feat_highway_footway,feat_highway_give_way,feat_highway_milestone,feat_highway_mini_roundabout,feat_highway_motorway_junction,feat_highway_path,feat_highway_platform,feat_highway_raceway,feat_highway_rest_area,feat_highway_services,feat_highway_speed_camera,feat_highway_steps,feat_highway_stop,feat_highway_street_lamp,feat_highway_toll_gantry,feat_highway_traffic_mirror,feat_highway_traffic_signals,feat_highway_traffic_signals;crossing,feat_highway_trailhead,feat_highway_turning_circle,feat_highway_turning_loop,feat_man_made_advertising,feat_man_made_antenna,feat_man_made_beacon,feat_man_made_bunker_silo,feat_man_made_cairn,feat_man_made_chimney,feat_man_made_communications_tower,feat_man_made_crane,feat_man_made_cross,feat_man_made_flagpole,feat_man_made_gasometer,feat_man_made_kiln,feat_man_made_lighthouse,feat_man_made_lookout_tower,feat_man_made_manhole,feat_man_made_mast,feat_man_made_monitoring_station,feat_man_made_pier,feat_man_made_pole,feat_man_made_reservoir,feat_man_made_reservoir_covered,feat_man_made_satellite_centre,feat_man_made_silo,feat_man_made_standpipe,feat_man_made_storage_tank,feat_man_made_street_cabinet,feat_man_made_surveillance,feat_man_made_survey_point,feat_man_made_tanks,feat_man_made_telescope,feat_man_made_threshing_floor,feat_man_made_tower,feat_man_made_utility_pole,feat_man_made_ventilation_shaft,feat_man_made_wastewater_plant,feat_man_made_water_tank,feat_man_made_water_tap,feat_man_made_water_tower,feat_man_made_water_well,feat_man_made_water_works,feat_man_made_watermill,feat_man_made_windmill,feat_man_made_windpump,feat_man_made_works,feat_man_made_yes,sum_points
0,80216819,6224.778569,45.208716,44.464286,28.000000,59.000000,2,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,80216858,2236.054168,79.563308,82.750000,71.750000,93.250000,2,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18
2,80216859,2138.725039,65.955069,67.333333,61.000000,75.000000,3,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17
3,80216883,2699.137580,32.332179,31.666667,19.000000,43.666667,3,6

## Preprocessing

### Duplicates

In [8]:
len(df) == len(df.drop_duplicates())

True

### Missing Values

In [9]:
df.isnull().sum().sort_values(ascending=False)

linkid                              0
feat_man_made_reservoir             0
feat_man_made_pier                  0
feat_man_made_monitoring_station    0
feat_man_made_mast                  0
                                   ..
feat_barrier_turnstile              0
feat_barrier_toll_gantry            0
feat_barrier_toll_booth             0
feat_barrier_swing_gate             0
sum_points                          0
Length: 106, dtype: int64

### Outliers

Outliers are data points which deviate from the rest of the data 

We have dealt with outliers by focusing on road segments where average velocity of vehicle traffic is between 0 and 180 km/h

### Feature Creation

In [10]:
max_speed_dict = {2:130,3:100,4:90,5:70,6:50,7:30}
df['Max_speed'] = df['Speed_Cat'].map(max_speed_dict)

In [11]:
# creating y target - over or underspeeding ?
df['y_target_speediff'] = df['Max_speed'] - df['Average_Velocity_of_Vehicle_Traffic']

### Feature Scaling

In [12]:
numerical = ['Daily_Average_Traffic_Intensity',
       'Average_Velocity_of_Vehicle_Traffic',
       'Median_of_velocity_of_Vehicle_Traffic',
       'FirstQuartil_of_velocity_of_Vehicle_Traffic',
       'ThirdQuartil_of_velocity_of_Vehicle_Traffic',
        'Max_speed', 'y_target_speediff' ]
categorical = ['Func_Class',
       'Speed_Cat']

In [24]:
remove_list = ['linkid','Func_Class','Speed_Cat']

In [25]:
num_col = list(df.columns)
num_col

['linkid',
 'Daily_Average_Traffic_Intensity',
 'Average_Velocity_of_Vehicle_Traffic',
 'Median_of_velocity_of_Vehicle_Traffic',
 'FirstQuartil_of_velocity_of_Vehicle_Traffic',
 'ThirdQuartil_of_velocity_of_Vehicle_Traffic',
 'Func_Class',
 'Speed_Cat',
 'feat_barrier_block',
 'feat_barrier_bollard',
 'feat_barrier_border_control',
 'feat_barrier_bump_gate',
 'feat_barrier_cattle_grid',
 'feat_barrier_chain',
 'feat_barrier_cycle_barrier',
 'feat_barrier_debris',
 'feat_barrier_door',
 'feat_barrier_entrance',
 'feat_barrier_fence',
 'feat_barrier_gate',
 'feat_barrier_height_restrictor',
 'feat_barrier_jersey_barrier',
 'feat_barrier_kerb',
 'feat_barrier_kissing_gate',
 'feat_barrier_lift_gate',
 'feat_barrier_rope',
 'feat_barrier_sally_port',
 'feat_barrier_security_check',
 'feat_barrier_stile',
 'feat_barrier_swing_gate',
 'feat_barrier_toll_booth',
 'feat_barrier_toll_gantry',
 'feat_barrier_turnstile',
 'feat_barrier_wall',
 'feat_barrier_yes',
 'feat_highway_bus_stop',
 'feat_

In [26]:
for col in remove_list:
    num_col.remove(col)

In [27]:
scaler = StandardScaler()
df_scl = df.copy()
for column in num_col:
    scaler.fit(df_scl[[column]])
    df_scl[column]=scaler.transform(df_scl[[column]]) 
df_scl

,linkid,Daily_Average_Traffic_Intensity,Average_Velocity_of_Vehicle_Traffic,Median_of_velocity_of_Vehicle_Traffic,FirstQuartil_of_velocity_of_Vehicle_Traffic,ThirdQuartil_of_velocity_of_Vehicle_Traffic,Func_Class,Speed_Cat,feat_barrier_block,feat_barrier_bollard,feat_barrier_border_control,feat_barrier_bump_gate,feat_barrier_cattle_grid,feat_barrier_chain,feat_barrier_cycle_barrier,feat_barrier_debris,feat_barrier_door,feat_barrier_entrance,feat_barrier_fence,feat_barrier_gate,feat_barrier_height_restrictor,feat_barrier_jersey_barrier,feat_barrier_kerb,feat_barrier_kissing_gate,feat_barrier_lift_gate,feat_barrier_rope,feat_barrier_sally_port,feat_barrier_security_check,feat_barrier_stile,feat_barrier_swing_gate,feat_barrier_toll_booth,feat_barrier_toll_gantry,feat_barrier_turnstile,feat_barrier_wall,feat_barrier_yes,feat_highway_bus_stop,feat_highway_crossing,feat_highway_crossing;traffic_signals,feat_highway_elevator,feat_highway_footway,feat_highway_give_way,feat_highway_milestone,feat_highway_mini_roundabout,feat_highway_motorway_junction,feat_highway_path,feat_highway_platform,feat_highway_raceway,feat_highway_rest_area,feat_highway_services,feat_highway_speed_camera,feat_highway_steps,feat_highway_stop,feat_highway_street_lamp,feat_highway_toll_gantry,feat_highway_traffic_mirror,feat_highway_traffic_signals,feat_highway_traffic_signals;crossing,feat_highway_trailhead,feat_highway_turning_circle,feat_highway_turning_loop,feat_man_made_advertising,feat_man_made_antenna,feat_man_made_beacon,feat_man_made_bunker_silo,feat_man_made_cairn,feat_man_made_chimney,feat_man_made_communications_tower,feat_man_made_crane,feat_man_made_cross,feat_man_made_flagpole,feat_man_made_gasometer,feat_man_made_kiln,feat_man_made_lighthouse,feat_man_made_lookout_tower,feat_man_made_manhole,feat_man_made_mast,feat_man_made_monitoring_station,feat_man_made_pier,feat_man_made_pole,feat_man_made_reservoir,feat_man_made_reservoir_covered,feat_man_made_satellite_centre,feat_man_made_silo,feat_man_made_standpipe,feat_man_made_storage_tank,feat_man_made_street_cabinet,feat_man_made_surveillance,feat_man_made_survey_point,feat_man_made_tanks,feat_man_made_telescope,feat_man_made_threshing_floor,feat_man_made_tower,feat_man_made_utility_pole,feat_man_made_ventilation_shaft,feat_man_made_wastewater_plant,feat_man_made_water_tank,feat_man_made_water_tap,feat_man_made_water_tower,feat_man_made_water_well,feat_man_made_water_works,feat_man_made_watermill,feat_man_made_windmill,feat_man_made_windpump,feat_man_made_works,feat_man_made_yes,sum_points,Max_speed,y_target_speediff
0,80216819,1.940874,-0.098961,-0.222566,-0.377877,-0.116365,2,6,-0.028973,-0.03052,0.0,0.0,0.0,0.0,0.0,-0.015191,0.0,-0.037231,0.0,-0.225706,0.0,0.0,0.0,0.0,-0.094982,0.0,-0.051704,0.0,0.0,-0.032785,-0.11783,-0.027114,-0.041526,-0.021486,-0.05044,-0.573013,-0.699251,0.0,-0.099452,0.0,-0.206672,-0.042346,-0.033983,-0.453036,0.0,0.0,0.0,-0.015191,-0.030392,-0.146317,0.0,-0.135151,-0.186002,-0.049278,-0.028428,1.895432,-0.064432,0.0,-0.143038,-0.035644,-0.041471,-0.021486,0.0,0.0,0.0,0.0,-0.014411,0.0,-0.018606,-0.04979,0.0,0.0,-0.010741,0.0,-0.01725,-0.081228,-0.021486,0.0,0.0,0.0,0.0,0.0,0.0,-0.038754,0.0,-0.074689,-0.036541,-0.066165,-0.018606,0.0,0.0,-0.05545,-0.032048,0.0,0.0,0.0,-0.061918,-0.028428,-0.021486,0.0,0.0,-0.033983,0.0,0.0,0.0,0.224824,-0.572187,-0.210940
1,80216858,-0.366965,0.578611,1.487709,1.665838,1.323577,2,4,-0.028973,-0.03052,0.0,0.0,0.0,0.0,0.0,-0.015191,0.0,-0.037231,0.0,-0.225706,0.0,0.0,0.0,0.0,8.664578,0.0,-0.051704,0.0,0.0,-0.032785,-0.11783,-0.027114,-0.041526,-0.021486,-0.05044,2.526479,0.647319,0.0,-0.099452,0.0,-0.206672,-0.042346,-0.033983,1.691893,0.0,0.0,0.0,-0.015191,-0.030392,-0.146317,0.0,5.636395,-0.186002,-0.049278,-0.028428,3.796455,-0.064432,0.0,-0.143038,-0.035644,-0.041471,-0.021486,0.0,0.0,0.0,0.0,-0.014411,0.0,-0.018606,-0.04979,0.0,0.0,-0.010741,0.0,-0.01725,-0.081228,-0.021486,0.0,0.0,0.0,0.0,0.0,0.0,-0.038754,0.0,-0.074689,-0.036541,-0.0661

columns=['Link_ID','Unnamed: 0','Daily_Average_Traffic_Intensity','Average_Velocity_of_Vehicle_Traffic',
       'Median_of_velocity_of_Vehicle_Traffic',
       'FirstQuartil_of_velocity_of_Vehicle_Traffic',
       'ThirdQuartil_of_velocity_of_Vehicle_Traffic', 'Func_Class',
       'Speed_Cat','geometry', 'points']

In [ ]:
columns=['Unnamed: 0','geometry']

In [ ]:
df2 = pd.read_csv('wdl_data/preprocessed_data')
df2

In [ ]:
df2.drop(columns=columns, inplace=True)

In [ ]:
raw_df = pd.merge(df2,df1, on = 'linkid')

In [ ]:
raw_df

In [ ]:
columns_to_drop = ['linkid','Average_Velocity_of_Vehicle_Traffic','Median_of_velocity_of_Vehicle_Traffic','FirstQuartil_of_velocity_of_Vehicle_Traffic',\
                  'ThirdQuartil_of_velocity_of_Vehicle_Traffic','Func_Class','Speed_Cat','Speed_Diff_Median','Max_speed'] 
final_data = raw_df.drop(columns=columns_to_drop)

In [ ]:
final_data

## Scaling

In [ ]:
#scaling features of Moritz
from sklearn.preprocessing import StandardScaler

In [ ]:
final_data.iloc[:,11:]

In [ ]:
scaler = StandardScaler()
final_data.iloc[:,11:] =  scaler.fit_transform(final_data.iloc[:,11:])

In [ ]:
final_data

## Modeling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate

In [ ]:
# Prepare X and y
X = final_data.drop(columns='Speed_Diff_Mean')
y = final_data['Speed_Diff_Mean']


X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3, 
                                                    random_state = 6) # Holdout


baseline_model = DummyRegressor(strategy="mean") # Baseline
baseline_model.fit(X_train, y_train) # Calculate value for stratgy
baseline_model.score(X_test, y_test) # Score model based on consistently predicting the strategy

In [ ]:
model = LinearRegression().fit(X_train, y_train) # instantiate and fit model 

model.score(X_test, y_test) # Score model

In [ ]:
cv_results = cross_validate(model, X, y, cv=5, 
                            scoring=['max_error',
                                     'r2', 
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error']
                           )
pd.DataFrame(cv_results) # Cross validation output

In [ ]:
cv_results['test_r2'].mean()# Cross validation results

## Dropping speed cat

In [ ]:
# Prepare X and y
X = final_data.drop(columns=['Speed_Diff_Mean','speed_2','speed_3','speed_4','speed_5','speed_6','speed_7'])
y = final_data['Speed_Diff_Mean']


X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3, 
                                                    random_state = 6) # Holdout


baseline_model = DummyRegressor(strategy="mean") # Baseline
baseline_model.fit(X_train, y_train) # Calculate value for stratgy
baseline_model.score(X_test, y_test) # Score model based on consistently predicting the strategy

In [ ]:
model = LinearRegression().fit(X_train, y_train) # instantiate and fit model 

model.score(X_test, y_test) # Score model

In [ ]:
cv_results = cross_validate(model, X, y, cv=5, 
                            scoring=['max_error',
                                     'r2', 
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error']
                           )
pd.DataFrame(cv_results) # Cross validation output

In [ ]:
cv_results['test_r2'].mean()# Cross validation results